<a href="https://colab.research.google.com/github/pardisbabaie/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:13 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [1,784 kB]
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports I

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2020-12-10 23:22:54--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1007KB/s    in 1.0s    

2020-12-10 23:22:56 (1007 KB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [5]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataviz.cnkhwlx8bf8m.us-east-2.rds.amazonaws.com:5432/Amazon_Vine"
config = {"user":"postgres", 
          "password": "parham1362", 
          "driver":"org.postgresql.Driver"}

In [7]:
vines = spark.read.jdbc(url=jdbc_url, table='vine_table', properties=config)
votes_df = vines.filter("total_votes > 20")
votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R3EZ0EPYLDA34S|          1|           14|         31|   N|                Y|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|
|R3GSK9MM8DNOYI|          1|            4|         32|   N|     

In [8]:
#votes above 50%
#helpful_votes_count = votes_df['helpful_votes']/votes_df['total_votes']
helpful_votes_df = votes_df.filter(votes_df['helpful_votes']/votes_df['total_votes']>.50)
helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|
|R10LZVBLQHBVJ0|          2|          151|        198|   N|                N|
|R1VR5GLGY1GE7N|          1|           49|         51|   N|     

In [9]:
# Paid Vine
paid_vine_df = helpful_votes_df.filter(helpful_votes_df['vine'] == "Y")
paid_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3KKUSGFZWSUIY|          5|           56|         63|   Y|                N|
|R10FO5UKKVZBK2|          3|           23|         23|   Y|                N|
|R11O4YSCPSNL6L|          3|           20|         26|   Y|                N|
| RM4KSGEOR7MU1|          5|           19|         24|   Y|                N|
| RG7VRMYLEXD23|          4|           22|         26|   Y|                N|
|R1OWK33OPI45KT|          5|           55|         58|   Y|                N|
|R1JRR530H4COA2|          5|           22|         28|   Y|                N|
| RQ5WD90PUNBU9|          5|           21|         24|   Y|                N|
|R12648VHCQWUV9|          4|           21|         28|   Y|                N|
|R286MFBAJ8NPD6|          5|           46|         51|   Y|     

In [10]:
#Unpaid Vine
unpaid_vine_df = helpful_votes_df.filter(helpful_votes_df['vine'] == "N")
unpaid_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|
|R10LZVBLQHBVJ0|          2|          151|        198|   N|                N|
|R1VR5GLGY1GE7N|          1|           49|         51|   N|     

In [11]:
review_total_paid_df = paid_vine_df.describe().show()

+-------+--------------+------------------+-----------------+------------------+----+-----------------+
|summary|     review_id|       star_rating|    helpful_votes|       total_votes|vine|verified_purchase|
+-------+--------------+------------------+-----------------+------------------+----+-----------------+
|  count|            90|                90|               90|                90|  90|               90|
|   mean|          null| 4.166666666666667|56.27777777777778|63.644444444444446|null|             null|
| stddev|          null|0.9858550158920684|66.20465847653779| 69.85704682607849|null|             null|
|    min|R10FO5UKKVZBK2|                 1|               16|                21|   Y|                N|
|    max| RYBLWBAL5M1AY|                 5|              347|               362|   Y|                N|
+-------+--------------+------------------+-----------------+------------------+----+-----------------+



In [12]:
review_total_unpaid_df = unpaid_vine_df.describe().show()

+-------+--------------+------------------+------------------+------------------+-----+-----------------+
|summary|     review_id|       star_rating|     helpful_votes|       total_votes| vine|verified_purchase|
+-------+--------------+------------------+------------------+------------------+-----+-----------------+
|  count|         37385|             37385|             37385|             37385|37385|            37385|
|   mean|          null|3.3699879630867997|50.030734251705226| 58.72266951986091| null|             null|
| stddev|          null|1.6354456245065196|121.92464486924389|132.14783078988978| null|             null|
|    min|R1005PFAPGMYQS|                 1|                11|                21|    N|                N|
|    max| RZZTUIX1574RC|                 5|             10498|             10780|    N|                Y|
+-------+--------------+------------------+------------------+------------------+-----+-----------------+



In [13]:
# Count paid vs unpaid reviews
review_paid = paid_vine_df.count()
review_unpaid = unpaid_vine_df.count()

print(review_paid)
print(review_unpaid)

90
37385


In [14]:
#Total reviews - both
total_reviews = review_unpaid + review_paid
print(total_reviews)

37475


In [19]:
#Number of 5-star reviews all
five_stars = vines.filter(vines["star_rating"] ==5).count()
print(five_stars)

1026924


In [16]:
#Count of 5-star paid vs unpaid
five_stars_review_paid = paid_vine_df[paid_vine_df["star_rating"]==5].count()
five_stars_review_unpaid = unpaid_vine_df[unpaid_vine_df["star_rating"]==5].count()

print(five_stars_review_paid)
print(five_stars_review_unpaid)

44
14626


In [20]:

#% of 5-star reviews for paid
paid_five_stars = ((five_stars_review_paid/five_stars)*100)
print(paid_five_stars)

0.004284640343394448


In [22]:

#% of 5-star reviews for unpaid
unpaid_five_stars = ((five_stars_review_unpaid/five_stars)*100)
print(unpaid_five_stars)

1.4242534014201635
